# Study of borrowers' reliability

Is this quick study we analize whether the family status or the children count of a client influence the fact of paying back the credit on time. The data used is client data from a bank - the statictics of client payement ability.


The results will be used to build a scoring model - the will automatically analyse data on clients to determine whether they are reliable or not.

In [143]:
import pandas as pd

pd.set_option('display.float_format', '{:,.2f}'.format)

df = pd.read_csv('bank_data.csv')

In [144]:
df.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,"-8,437.67",42,college,0,married,0,F,employee,0,"253,875.64",home purchase
1,1,1,"-4,024.80",36,high school,1,married,0,F,employee,0,"112,080.01",car purchase
2,2,0,"-5,623.42",33,NaN,1,married,0,M,employee,0,"145,885.95",home purchase
3,3,3,"-4,124.75",32,high school,1,married,0,M,employee,0,"267,628.55",extra education
4,4,0,"340,266.07",53,high school,1,cohabitant,1,F,retired,0,"158,616.08",organize wedding


### Preprocessing

The data has to be processed as we have empty values. Let's explore it further

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         19390 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           19319 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


empty values are found in 'days_employed' and 'total_income'. Let's check the proportion of missing values

In [146]:
df.isna().mean()

Unnamed: 0         0.00
children           0.00
days_employed      0.10
dob_years          0.00
education          0.10
education_id       0.00
family_status      0.00
family_status_id   0.00
gender             0.00
income_type        0.00
debt               0.00
total_income       0.10
purpose            0.10
dtype: float64

10% of empty values. Just deleting them would distort our distribution and may influence our further analisys negatively. Let's study the column 'days_employed'

In [147]:
df.days_employed.describe()

count    19,351.00
mean     63,046.50
std     140,827.31
min     -18,388.95
25%      -2,747.42
50%      -1,203.37
75%        -291.10
max     401,755.40
Name: days_employed, dtype: float64

These negative values must be due to an error of some kind, let's remove them.

In [148]:
df.days_employed = df.days_employed.apply(lambda n: -n if n<0 else n)
df.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,"8,437.67",42,college,0,married,0,F,employee,0,"253,875.64",home purchase
1,1,1,"4,024.80",36,high school,1,married,0,F,employee,0,"112,080.01",car purchase
2,2,0,"5,623.42",33,NaN,1,married,0,M,employee,0,"145,885.95",home purchase
3,3,3,"4,124.75",32,high school,1,married,0,M,employee,0,"267,628.55",extra education
4,4,0,"340,266.07",53,high school,1,cohabitant,1,F,retired,0,"158,616.08",organize wedding


There are still some issues with the <code> days employed </code> column, a values like 340,266.07 can't be correct

In [149]:
df.days_employed.describe()

count    19,351.00
mean     66,914.73
std     139,030.88
min          24.14
25%         927.01
50%       2,194.22
75%       5,537.88
max     401,755.40
Name: days_employed, dtype: float64

We see that <b> maximum = 401755 days of work. </b > Check how many years it is: 401755/365 = 1100.7: more than a thousand years! This is clearly a mistake and something needs to be done with such data. Maybe instead of days, people entered the number of hours? Check: 1100.7/24 = 45.8. If a person is aged, it is possible that he worked for 45 years. Let's check what category of employment these data belong to.

In [150]:
df[df['days_employed'] > 20000]['income_type'].value_counts()

retired       3443
unemployed       2
Name: income_type, dtype: int64

It became clear that mainly pensioners fall into our category. Probably their experience was recorded in hours. Let's this by applying a function to the column.

In [151]:
df.days_employed = df.days_employed.apply(lambda n: n/24 if n > 20000 else n)
df.days_employed.describe()

count   19,351.00
mean     4,641.64
std      5,355.96
min         24.14
25%        927.01
50%      2,194.22
75%      5,537.88
max     18,388.95
Name: days_employed, dtype: float64

We see that now we have a maximum value of 18388.95 and if you calculate the value in years it turns out 18389/365 = 50 years, which is quite plausible. Now let's consider the column'dob_years'

First we deal with all the empty values

In [152]:
df[df['days_employed'].isna()]['income_type'].value_counts()

employee               1105
companion               508
retired                 413
government employee     147
entrepreneur              1
Name: income_type, dtype: int64

In [153]:
df[df['total_income'].isna()]['income_type'].value_counts()

employee               1105
companion               508
retired                 413
government employee     147
entrepreneur              1
Name: income_type, dtype: int64

We see that people without the specified income and seniority of work have different types of occupation and most likely an error occurred when entering data. We fill in the empty values of work experience by the average number of work experience of people of the same age category. First we will create several age categories. Before we do this, make sure that in the column 'dob_years' everything is in order

In [154]:
df.dob_years.describe()

count   21,525.00
mean        43.29
std         12.57
min          0.00
25%         33.00
50%         42.00
75%         53.00
max         75.00
Name: dob_years, dtype: float64

Let's deal with empty values by filling them correspondingly to the mean of matching category

In [155]:
occupation_cat = df.groupby('income_type').agg({'dob_years': 'mean'}).reset_index()
occupation_cat

,income_type,dob_years
0,companion,39.70
1,employee,39.82
2,entrepreneur,42.50
3,government employee,40.64
4,on maternity leave,39.00
5,retired,59.06
6,student,22.00
7,unemployed,38.00


In [156]:
occupation_cat['dob_years'] = occupation_cat['dob_years'].astype(int)
age_map_dict = {row['income_type']: row['dob_years'] for index, row in occupation_cat.iterrows()}

df['dob_years'] = df['dob_years'].fillna(df['income_type'].map(age_map_dict))

df.dob_years.describe()

count   21,525.00
mean        43.29
std         12.57
min          0.00
25%         33.00
50%         42.00
75%         53.00
max         75.00
Name: dob_years, dtype: float64

Let's now create age groups that will help us fill the empty values in the 'days_employed' column

In [157]:
def dob_years_cat(value):
    if value < 25:
        return "young"
    elif 25 <= value <= 40:
        return "adult"
    elif 40 < value < 65:
        return "middle age"
    return "elderly"

df['dob_years_cat'] = df.dob_years.apply(dob_years_cat)
age_cat = df.groupby('dob_years_cat').agg({'days_employed': 'mean'}).reset_index()
age_cat

,dob_years_cat,days_employed
0,adult,"1,936.37"
1,elderly,"13,636.41"
2,middle age,"6,314.09"
3,young,"1,262.06"


We got the average number of years of service according to the age category. Now fill in the empty values in the 'days _ employed' column

In [158]:
days_map_dict = {row['dob_years_cat']: row['days_employed'] for index, row in age_cat.iterrows()}
df['days_employed'] = df['days_employed'].fillna(df['dob_years_cat'].map(days_map_dict))


Let's replace the float type with the int data type in the 'days_employed' column because it specifies the number of days.

In [159]:
df.days_employed = df.days_employed.astype(int)

With missing values in 'income _ type' we will do the same only instead of age, we will rely on education. We already have the ivision into levels of education we have only to do something about duplicates that will prevent us from further analysis

Now we will process the column "education"

In [160]:
df.education.value_counts()

high school             13750
college                  4718
college (unfinished)      668
primary                   250
PhD                         4
Name: education, dtype: int64

We can already fill in the missing data in the column'total_income' based on education. This time we use the median so that extreme values do not greatly affect the average salary

In [161]:
education_salary_median = df.groupby('education')['total_income'].median()
education_salary_median 

education
PhD                    185,687.12
college                175,625.31
college (unfinished)   159,780.92
high school            136,643.35
primary                117,314.52
Name: total_income, dtype: float64

In [162]:
for value in education_salary_median:
    for index in education_salary_median:
        df.loc[(df['total_income'].isna()) & (df['education'] == index), 'total_income'] = value


In [176]:
df = df.dropna(axis=0)

In [180]:
df.family_status.unique()


array(['married', 'cohabitant', 'widow', 'single', 'divorced'],
      dtype=object)

In [181]:
df.gender.unique()

array(['F', 'M', 'XNA'], dtype=object)

In [182]:
df.groupby('gender')['debt'].agg(['count', 'mean'])

,count,mean
gender,,
F,10313,0.07
M,5297,0.11
XNA,1,0.00


In [183]:
df.duplicated().sum()

0

In [184]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [185]:
df.children.describe()

count   15,611.00
mean         0.48
std          0.75
min          0.00
25%          0.00
50%          0.00
75%          1.00
max          5.00
Name: children, dtype: float64

In [186]:
df.children.value_counts()

0    10235
1     3565
2     1545
3      233
4       26
5        7
Name: children, dtype: int64

In [188]:
df.children = df.children.apply(lambda n: abs(n))
df.children = df.children.apply(lambda n: 2 if n == 20 else n)
df.children.value_counts()

0    10235
1     3565
2     1545
3      233
4       26
5        7
Name: children, dtype: int64

### Credit purpose lemmatisation

To simplify the task of reading and understanding data, credit goals in the'pupose 'column need to be grouped by topic. For this, we will first find keywords. In order to do this, we write a function that will accept the entire column as the input.

In [189]:
!pip3 install pymystem3
from pymystem3 import Mystem
from collections import Counter 

m = Mystem()

def lemmatise(column):
    lemmas = []
    purpose_lemmatized = [m.lemmatize(phrase) for phrase in column]
    for item in purpose_lemmatized:
        for word in item:
            lemmas.append(word)  
    return lemmas

words_lemmatized = lemmatise(df.purpose)
lemmas_dict = Counter(words_lemmatized)
lemmas_dict

Counter({'home': 1022,
         ' ': 29206,
         'purchase': 1297,
         '\n': 9398,
         'car': 2754,
         'extra': 369,
         'education': 2827,
         'organize': 627,
         'wedding': 1881,
         'Housing': 525,
         'operations': 525,
         'buying': 3099,
         'family': 521,
         'housing': 1577,
         ' \n': 6213,
         'real': 4654,
         'estate': 4654,
         'commercial': 1071,
         'residential': 495,
         'construction': 1020,
         'of': 907,
         'own': 505,
         'for': 929,
         'the': 402,
         'a': 402,
         'used': 792,
         'transactions': 1579,
         'cars': 371,
         'to': 1075,
         'engage': 315,
         'in': 315,
         'receive': 350,
         'deal': 768,
         'obtaining': 358,
         'additional': 358,
         'your': 1297,
         'higher': 730,
         'rent': 527,
         'buy': 410,
         'specific': 337})

We got a big dictionary containing separate words. You can thus make a list of the main goals of the loan.

In [194]:
purposes = ['home', 'property', 'wedding', 'car', 'education']

In [196]:
def column_creation(row):
    for word in m.lemmatize(row):
        if word in purposes:
            return word

df['purpose_lemmatized'] = df.purpose.apply(column_creation)
df.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_cat,purpose_lemmatized
0,0,1,8437,42,college,0,married,0,F,employee,0,"253,875.64",home purchase,middle age,home
1,1,1,4024,36,high school,1,married,0,F,employee,0,"112,080.01",car purchase,adult,car
2,3,3,4124,32,high school,1,married,0,M,employee,0,"267,628.55",extra education,adult,education
3,4,0,14177,53,high school,1,cohabitant,1,F,retired,0,"158,616.08",organize wedding,middle age,wedding
4,5,0,926,27,college,0,cohabitant,1,M,companion,0,"255,763.57",home purchase,adult,home


### Data analysis

Let's now begin to answer the question about borrowers' reliability.

#### 1. Is there a dependency between the number of children and being in/out of debt?

In [197]:
df.groupby('children')['debt'].agg(['count', 'mean'])

,count,mean
children,,
0,10235,0.08
1,3565,0.10
2,1545,0.10
3,233,0.08
4,26,0.08
5,7,0.00


Let's separate the clients in those who have and those who have no kids

In [198]:
df['no_children'] = (df.children == 0) * 1
df.groupby('no_children')['debt'].agg(['count', 'mean'])

,count,mean
no_children,,
0,5376,0.10
1,10235,0.08


More debt among people with children. Let's see if education is of nay influence.

In [199]:
df.groupby('education')['debt'].agg(['count', 'mean'])

,count,mean
education,,
PhD,3,0.00
college,3794,0.06
college (unfinished),548,0.10
high school,11063,0.09
primary,203,0.13


In [201]:
df['university_degree'] = df.education.isin(['college', 'PhD'])*1
df.groupby('university_degree')['debt'].agg(['count', 'mean'])

,count,mean
university_degree,,
0,11814,0.09
1,3797,0.06


A lot less debt among people with a college degree

In [202]:
df['score'] = df['university_degree'] + df['no_children']
df.groupby('score')['debt'].agg(['count', 'mean'])

,count,mean
score,,
0,3947,0.11
1,9296,0.08
2,2368,0.05


We received three customer categories:
1. Customers who have children and who have not graduated from high school - for them the probability of not repaying the debt: 11%
2. Clients without children and higher education or with children and higher education: probability of debt: 8%
3. Customers who do not have children and who have graduated from high school - for them the probability of not repaying the debt is only 5%

#### 2. Is there a dependancy between the family status and debt?

In [203]:
df.groupby('family_status')['debt'].agg(['count','mean'])

,count,mean
family_status,,
cohabitant,3182,0.09
divorced,847,0.07
married,8891,0.08
single,2007,0.10
widow,684,0.06


there is more debt between the single and the unmarried

In [205]:
df['was_not_married'] = df.family_status.isin(['single', 'cohabitant']) * 1
df.groupby('was_not_married')['debt'].agg(['count', 'mean'])

,count,mean
was_not_married,,
0,10422,0.08
1,5189,0.10


In [208]:
df.groupby('dob_years_cat')['debt'].agg(['count', 'mean'])

,count,mean
dob_years_cat,,
adult,6265,0.10
elderly,662,0.05
middle age,7993,0.07
young,691,0.10


With age, customer reliability grows. To include age in our scoring create an additional column of normalized age and then add it to our scoring column

In [209]:
df['dob_years_normalized'] = df['dob_years'] / df['dob_years'].max()
df[df['dob_years_normalized'] < 0.3].head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_cat,purpose_lemmatized,no_children,university_degree,score,was_not_married,dob_years_normalized
18,23,0,272,21,college,0,cohabitant,1,M,employee,0,"128,265.72",organize wedding,young,wedding,1,1,3,1,0.28
32,42,0,1257,20,college (unfinished),2,single,4,F,employee,0,"82,065.09",to receive education,young,education,1,0,2,1,0.27
109,149,0,2664,0,high school,1,divorced,3,F,employee,0,"70,176.44",Housing operations,young,None,1,0,1,0,0.00
113,153,1,132,20,high school,1,married,0,M,companion,1,"134,750.75",housing,young,None,0,0,0,0,0.27
155,208,0,749,22,high school,1,cohabitant,1,M,companion,0,"162,492.02",specific education,young,education,1,0,2,1,0.29


The higher the indicator - the higher the score

In [210]:
df['score'] = df['score'] + df['dob_years_normalized']
df['score_quart'] = pd.qcut(df['score'], 5)
df.groupby('score_quart')['debt'].agg(['count', 'mean'])

,count,mean
score_quart,,
"(-0.001, 1.333]",3124,0.10
"(1.333, 1.6]",3211,0.10
"(1.6, 1.827]",3067,0.06
"(1.827, 2.587]",3089,0.09
"(2.587, 3.96]",3120,0.06


Thus, we divided our scoring into 5 parts having received different categories of customers, at the beginning of the table - the most unreliable and below - vice versa

In [211]:
df['score_rounded'] = df['score'].round()
df.groupby('score_rounded')['debt'].agg(['mean', 'count'])

,mean,count
score_rounded,,
0.00,0.11,1474
1.00,0.10,3417
2.00,0.08,6870
3.00,0.06,3389
4.00,0.06,461


Now it is quite easy to determine the reliability of the borrower by scoring

#### 3. Is there a relationship between income level and loan repayment on time?

It remains to see how the income level affects the probability of debt

In [212]:
df['total_income_q'] = pd.qcut(df['total_income'], 5)
df.groupby('total_income_q')['debt'].agg(['mean', 'count'])

,mean,count
total_income_q,,
"(20667.263, 94868.486]",0.08,3123
"(94868.486, 128418.675]",0.09,3122
"(128418.675, 164320.213]",0.09,3122
"(164320.213, 223383.426]",0.09,3122
"(223383.426, 2200852.21]",0.07,3122


This time, the data shows a non-linear relationship. People with low incomes (up to about 130,000) are less prone to debt - 8% than people whose income is between 130 and 160 thousand - 9%.

Starting with an income of 160,000, the higher the income the less arrears.

8% - among those with income between 160,000 and 214,000 and
7% - among those with an income above 214 thousand

#### 3. How do different goals of a loan affect its return on time?

In [213]:
df.groupby('purpose_lemmatized')['debt'].agg(['count', 'mean'])

,count,mean
purpose_lemmatized,,
car,2754,0.10
education,2827,0.09
home,1022,0.07
wedding,1881,0.07


We see that the purpose of a loan affects the percentage of debts, most of the debt shows when the pupropse is to buy a car.
9% for education
7% of debt on loans for the purchase of house or organising a wedding

### 4. Conclusion

- People who have children with a probability of 9% will not repay the loan on time while people without children will not repay the loan with a probability of 8%


- People who have never been married have a better chance of not returning money on time - 9-10% Although people married as well as divorced or widows/widowers will not repay the loan with a probability of 7-8%


- Salary also affects credit repayment on time. People with low incomes (up to about 130,000) are less prone to debt - 8% than people whose income is between 130 and 160 thousand - 9%. Starting with an income of 160,000, the higher the income the less arrears. 8% - among those with income between 160,000 and 214,000 and 7% - among those with income above 214 thousand.


- We see that the purpose of the loan affects the percentage of arrears, the larger the goal the less arrears. 8-9% of loans for weddings, car purchases and education. 7% of debt on loans for the purchase of housing or real estate


- Education also affects the probability of debt: 9% of the chance not to return money for those who have not graduated from university and only 5% for those who have completed higher education.


- The age of the client also affects its reliability. The older the client, the more reliable he is. Up to forty years, the probability of not returning the loan is 10%, while among pensioners of debtors only 5% and among people aged between 40 and 65 - only 7%

<code> The ideal client </code> will be a person who has graduated from a university, who for 40 is married, without children, with an income of either up to 130,000 or above 214,000 who need money for real estate or housing.


<code> The most unreliable </code> client is a person who has not completed a university, who is under 40 years old (the less unreliable) with a salary of between 130 and 214 thousand with the purpose of a loan such as buying a car or paying for an education that has children and has never been married

